In [1]:
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
import scipy.sparse as sp


/home/dzakirm/anaconda3/envs/satdat2024/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-24 20:58:10.391602: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 20:58:10.453858: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-24 20:58:11.410532: W tensorflow/compiler/tf2tensorrt/utils/p

In [2]:
df = pd.read_parquet("../data/processed/arxiv-abstracts-cleaned_v2.parquet", columns=["id", "authors","title_clean", "abstract_clean"])

client = QdrantClient(host="localhost", port=6333)

collection_count = "arxiv_count_vector"
collection_tfidf = "arxiv_tfidf"
collection_embedding = "arxiv_embedding"

# sparse_cv = sp.load_npz("../data/processed/count_vectorizer.npz")
# sparse_tfidf = sp.load_npz("../data/processed/tfidf_vectorizer.npz")
#
# dense_cv = sparse_cv.toarray()
# dense_tfidf = sparse_tfidf.toarray()

In [3]:
df

,id,authors,title_clean,abstract_clean
0,1606.02306,Shahar Hod,the superradiant instability regime of the spi...,spinning kerr black holes are known to be supe...
1,hep-ph/9812388,"W.M. Alberico, M.B. Barbaro, S.M. Bilenky, J.A...",strange form factors of the proton a new analy...,we consider ratios of elastic neutrinoantineut...
2,astro-ph/0208171,"M. Boettcher (Rice Univ. / Ohio Univ.), R. Muk...",predictions of the highenergy emission from bl...,spectral fitting of the radio through hard xra...
3,0912.2791,"Dong-Keun Ki, Seung-Geol Nam, Hu-Jong Lee, and...",dependence of quantumhall conductance on the e...,by using fourterminal configurations we invest...
4,0707.3555,"A. Russell, Vladimir I. Fal'ko",power dependence of the photocurrent lineshape...,we propose a kinetic theory to describe the po...
...,...,...,...,...
499995,2101.12190,"Xuanqiang Zhao, Benchi Zhao, Zihe Wang, Zhixin...",practical distributed quantum information proc...,distributed quantum information processing is ...
499996,1611.06158,"Manuel G\""unther and Andras Rozsa and Terrance...",affact alignmentfree facial attribute classifi...,facial attributes are softbiometrics that allo...
499997,1907.06113,Yiannis Loizides,quasipolynomials and the singular theorem,in this short note we revisit the shiftdesingu...
499998,cond-mat/9902090,"I. Knittel, F. Gagel, M. Schreiber",quantum transport and momentum conserving deph...,we study numerically the influence of momentum...


In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2', cache_folder="./model_cache")
sample_text = df.iloc[0]['title_clean'] + ". " + df.iloc[0]['abstract_clean']
sample_embedding = model.encode(sample_text)
dim_embedding = len(sample_embedding)


/home/dzakirm/anaconda3/envs/satdat2024/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# client.create_collection(
#     collection_name=collection_count,
#     vectors_config=VectorParams(size=dim_count, distance=Distance.COSINE)
# )
# client.create_collection(
#     collection_name=collection_tfidf,
#     vectors_config=VectorParams(size=dim_tfidf, distance=Distance.COSINE)
# )
print(dim_embedding)
client.create_collection(
    collection_name=collection_embedding,
    vectors_config=VectorParams(size=dim_embedding, distance=Distance.COSINE)
)


384


True

In [6]:
# points_count = []
# points_tfidf = []
points_embedding = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    doc_id = row['id']
    authors = row['authors']
    title = row['title_clean']
    abstract = row['abstract_clean']

    text = title + ". " + abstract
    embedding_vector = model.encode(text)

    # # Titik untuk CountVectorizer
    # point_count = {
    #     "id": int(idx),
    #     "vector": row['count_vector'],  # Pastikan berupa list numerik
    #     "payload": {
    #         "title": title,
    #         "abstract": abstract,
    #     }
    # }
    # # Titik untuk TF-IDF
    # point_tfidf = {
    #     "id": int(idx),
    #     "vector": row['tfidf'],  # Pastikan berupa list numerik
    #     "payload": {
    #         "title": title,
    #         "abstract": abstract,
    #     }
    # }

    # Titik untuk embedding all-MiniLM
    point_embedding = {
        "id": int(idx),
        "vector": embedding_vector.tolist(),
        "payload": {
            "id": str(doc_id),
            "authors": authors,
            "title": title,
            "abstract": abstract,
        }
    }

    # points_count.append(point_count)
    # points_tfidf.append(point_tfidf)
    points_embedding.append(point_embedding)



100%|██████████| 500000/500000 [57:34<00:00, 144.73it/s] 


In [7]:
batch_size = 100

def upsert_batches(collection, points):
    for i in tqdm(range(0, len(points), batch_size)):
        batch = points[i:i+batch_size]
        client.upsert(collection_name=collection, points=batch)

# upsert_batches(collection_count, points_count)
# upsert_batches(collection_tfidf, points_tfidf)
upsert_batches(collection_embedding, points_embedding)


100%|██████████| 5000/5000 [03:06<00:00, 26.77it/s]
